# Version Control with Icechunk

In [1]:
import zarr
from icechunk import Repository, StorageConfig

## Create a new Zarr store backed by Icechunk

This example uses an in-memory store.

In [2]:
repo = Repository.create(
    storage=StorageConfig.memory("test")
)
repo

1. Why not checkout main by default?
2. Why can I create snapshots on the `None` branch

## Snaphotting

### Concepts

1. `store.commit` creates a _snapshot_ of the data.
2. Every snapshot is associated with a _snapshot ID_.
3. Use the _snapshot ID_ to time-travel within your data's history.

### Create a snapshot

In [3]:
session = repo.writable_session("main")
store = session.store()
root_group = zarr.group(store=store)

In [4]:
root_group.attrs["attr"] = "first_attr"

In [5]:
first_commit = session.commit("first commit")
first_commit

'23G8HS855AW4W7E5QX20'

In [6]:
dict(root_group.attrs)

{'attr': 'first_attr'}

In [7]:
session = repo.writable_session("main")
store = session.store()
root_group = zarr.group(store=store)

root_group.attrs["attr"] = "second_attr"
second_commit = session.commit("second commit")
second_commit

'5YQ4CHFRTA8WY621RREG'

### View the current snapshot ID

In [8]:
repo.lookup_branch("main")

'5YQ4CHFRTA8WY621RREG'

### Time-travel to a snapshot

Here's where we are:

In [9]:
session.snapshot_id, dict(root_group.attrs)

('5YQ4CHFRTA8WY621RREG', {'attr': 'second_attr'})

In [11]:
session = repo.readonly_session(snapshot_id=first_commit)
root_group = zarr.open_group(store=store, mode="r")
dict(root_group.attrs)

{'attr': 'second_attr'}

### Snapshotting is only allowed at the tip of a branch

TODO: need better error message

In [13]:
try:
    root_group.attrs["attr"] = "will_fail"
    session.commit("this should fail")
except Exception as e:
    print(e)

store error: cannot write to read-only store


## Branching

### Create a new branch

We will create a new branch starting at `first_commit`

In [14]:
repo.create_branch("feature", first_commit)

In [15]:
assert repo.lookup_branch("feature") == first_commit

In [16]:
session = repo.writable_session("feature")
store = session.store()

root_group = zarr.group(store=store)
dict(root_group.attrs)

{'attr': 'first_attr'}

In [17]:
root_group.attrs["attr"] = "new_branch_attr"
new_branch_commit = session.commit("commit on new branch")

## Tagging

### Creating a new tag

In [18]:
repo.create_tag("v1", snapshot_id=first_commit)

In [19]:
repo.create_tag("v2", snapshot_id=second_commit)

### Time-travel to a tag

Pass the `tag` argument to `checkout`

In [20]:
session = repo.readonly_session(tag="v1")
session.snapshot_id

'23G8HS855AW4W7E5QX20'